<a href="https://colab.research.google.com/github/nitish-01/Deep-Learning/blob/main/DL_model_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import gc;
gc.collect()
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7412 sha256=0f507e070d9bab4bef54085991769b543d501bc4be41012dcd75b8a1fbb70cf9
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 111.3 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [ ]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')
datapath = "content/drive/My Drive/Colab Notebooks/data/cse7512-00-dl-tp-pt1/"

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/Colab Notebooks/data/cse7512-00-dl-tp-pt1.zip'

Streaming output truncated to the last 5000 lines.
  inflating: train/airplane/10.bmp   
  inflating: train/airplane/1006.bmp  
  inflating: train/airplane/1012.bmp  
  inflating: train/airplane/103.bmp  
  inflating: train/airplane/1030.bmp  
  inflating: train/airplane/1037.bmp  
  inflating: train/airplane/1039.bmp  
  inflating: train/airplane/104.bmp  
  inflating: train/airplane/1055.bmp  
  inflating: train/airplane/1056.bmp  
  inflating: train/airplane/106.bmp  
  inflating: train/airplane/1089.bmp  
  inflating: train/airplane/1090.bmp  
  inflating: train/airplane/1106.bmp  
  inflating: train/airplane/1117.bmp  
  inflating: train/airplane/1124.bmp  
  inflating: train/airplane/1131.bmp  
  inflating: train/airplane/1136.bmp  
  inflating: train/airplane/1152.bmp  
  inflating: train/airplane/1153.bmp  
  inflating: train/airplane/1170.bmp  
  inflating: train/airplane/1182.bmp  
  inflating: train/airplane/119.bmp  
  inflating: train/airplane/12.bmp   
  inflating: train/

In [ ]:
import torch
import torch.nn as nn
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import sqrt
from torchvision import datasets, transforms


class FlattenBatch(nn.Module):
    def __init__(self):
        super(FlattenBatch, self).__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)


class LowRankLayer(nn.Module):
    def __init__(self, input_size, output_size=2 ** 8, d=8, K=2, pi_size=28, adaptive=True):
        """
        d: rank of decompositions
        K: number of pairs (U(k), V(k))
        pi_size: size of pi function hidden layer
        """
        super(LowRankLayer, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.d = d
        self.K = K
        self.pi_size = pi_size
        self.adaptive = adaptive

        self.Vs1 = [nn.Parameter(torch.Tensor(self.input_size, self.d), requires_grad=True) for k in range(self.K)]
        self.Us1 = [nn.Parameter(torch.Tensor(self.d, self.output_size), requires_grad=True) for k in range(self.K)]
        for k in range(self.K):
            nn.init.kaiming_uniform_(self.Vs1[k], a=2)
            nn.init.kaiming_uniform_(self.Us1[k], a=2)
            self.register_parameter('U{}'.format(k), self.Us1[k])
            self.register_parameter('V{}'.format(k), self.Vs1[k])

        if self.adaptive:
            self.W_pi = nn.Parameter(torch.Tensor(self.pi_size, self.K), requires_grad=True)
            nn.init.kaiming_uniform_(self.W_pi, a=2)
        else:
            self.W_pi = None

    def forward(self, x):
        # x has shape (n_samples, channel_count, input_size)
        if len(x.shape) > 3:
            x = x.view(x.shape[0], x.shape[1], -1)

        if self.adaptive:
            pool_size = x.shape[2] // self.pi_size
            x_pooled = F.avg_pool1d(x, pool_size)  # (n_samples, channel_count, pi_size)
            pi = x_pooled.matmul(self.W_pi)  # (n_samples, channel_count, K)
        else:
            pi = torch.full((x.shape[0], x.shape[1], self.K), 1 / self.K)

        pi = torch.sigmoid(pi)

        Wx = [(x.matmul(self.Vs1[k])).matmul(self.Us1[k]) for k in range(self.K)]
        Wx = torch.stack(Wx, dim=2)  # (n_samples, channel_count, K, output_size)
        Wx = (pi.view(*pi.shape, 1) * Wx).sum(dim=2)  # (n_samples, channel_count, output_size)

        return Wx
        

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        assert len(x.shape) == 4
        x = x.view(x.shape[0], x.shape[1], -1)  # flattens (w,h) -> w*h
        x = x.permute(0, 2, 1)  # permutes 1'st and 2'nd dimension, i.e. move n_channels to the last axis
        return x


class Unflatten(nn.Module):
    def __init__(self):
        super(Unflatten, self).__init__()

    def forward(self, x):
        assert len(x.shape) == 3
        x = x.permute(0, 2, 1)
        w = h = int(sqrt(x.shape[2]))
        x = x.view(x.shape[0], x.shape[1], w, h)
        return x


class MobileNet_CIFAR_LowRank(nn.Module):
    def __init__(self, d=8, K=2, pi_size=8):
        super(MobileNet_CIFAR_LowRank, self).__init__()

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, kernel_size=3, stride=stride, padding=1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),

                Flatten(),
                LowRankLayer(inp, oup, d, K, pi_size, adaptive=True),
                Unflatten(),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),
            conv_dw(32, 32, 1),
            conv_dw(32, 32, 1),
            conv_dw(32, 64, 1),

            conv_dw(64, 64, 2),
            conv_dw(64, 64, 2),
            conv_dw(64, 64, 2),
            conv_dw(64, 128, 2),

            conv_dw(128, 128, 2),
            conv_dw(128, 128, 2),
            conv_dw(128, 128, 2),
            conv_dw(128, 256, 2),
            nn.AvgPool2d(8, ceil_mode=True, count_include_pad=True),
        )
        self.fc = nn.Linear(256, 10)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 256)
        x = self.fc(x)
        return x

def Model():
    r"""Return your custom model
    """
    return MobileNet_CIFAR_LowRank()

In [ ]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
SAVEPATH = '/content/drive/My Drive/Colab Notebooks/data/'
WEIGHTDECAY = 5e-4
MOMENTUM = 0.9
BATCHSIZE = 64
LR = 0.1
EPOCHS = 50
PRINTFREQ = 10

In [ ]:
import time

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


def main():
    model = Model()

    optimizer = torch.optim.SGD(model.parameters(), lr=LR,
                                momentum=MOMENTUM, weight_decay=WEIGHTDECAY,
                                nesterov=True)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [25, 40],
                                                     gamma=0.1)
    criterion = torch.nn.CrossEntropyLoss()

    model = model.cuda()
    criterion = criterion.cuda()

    transform = transforms.Compose([
        transforms.Pad(4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
        transforms.ToTensor()])
    
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    train_transform = transforms.Compose([
        transforms.RandomCrop(96, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

    train_dataset = torchvision.datasets.ImageFolder(
        './train', transform=train_transform)
    train_loader = DataLoader(train_dataset,
                              batch_size=BATCHSIZE, shuffle=True,
                              num_workers=4, pin_memory=True)

    last_top1_acc = 0
    for epoch in range(EPOCHS):
        print("\n----- epoch: {}, lr: {} -----".format(
            epoch, optimizer.param_groups[0]["lr"]))

        # train for one epoch
        start_time = time.time()
        last_top1_acc = train(train_loader, epoch, model, optimizer, criterion)
        elapsed_time = time.time() - start_time
        print('==> {:.2f} seconds to train this epoch\n'.format(
            elapsed_time))

        # learning rate scheduling
        scheduler.step()

        # Save model each epoch
        torch.save(model.state_dict(), SAVEPATH+'model_weightz.pth')

    print(f"Last Top-1 Accuracy: {last_top1_acc}")


def train(train_loader, epoch, model, optimizer, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, data_time, losses,
                             top1, top5, prefix="Epoch: [{}]".format(epoch))
    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input = input.cuda()
        target = target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss, accuracy 
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0].item(), input.size(0))
        top5.update(acc5[0].item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINTFREQ == 0:
            progress.print(i)

    print('=> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))
    return top1.avg


if __name__ == "__main__":
    main()


----- epoch: 0, lr: 0.1 -----
Epoch: [0][ 0/79]	Time  1.507 ( 1.507)	Data  0.577 ( 0.577)	Loss 2.4255e+00 (2.4255e+00)	Acc@1  10.94 ( 10.94)	Acc@5  51.56 ( 51.56)
Epoch: [0][10/79]	Time  2.362 ( 2.286)	Data  0.000 ( 0.053)	Loss 2.5051e+00 (2.4852e+00)	Acc@1  14.06 ( 11.51)	Acc@5  59.38 ( 53.12)
Epoch: [0][20/79]	Time  2.361 ( 2.324)	Data  0.000 ( 0.028)	Loss 2.4153e+00 (2.4408e+00)	Acc@1  15.62 ( 12.57)	Acc@5  57.81 ( 56.92)
Epoch: [0][30/79]	Time  2.361 ( 2.336)	Data  0.000 ( 0.019)	Loss 2.1784e+00 (2.3955e+00)	Acc@1  17.19 ( 13.76)	Acc@5  78.12 ( 59.83)
Epoch: [0][40/79]	Time  2.365 ( 2.343)	Data  0.000 ( 0.014)	Loss 2.3265e+00 (2.3516e+00)	Acc@1  17.19 ( 14.52)	Acc@5  65.62 ( 62.04)
Epoch: [0][50/79]	Time  2.365 ( 2.347)	Data  0.000 ( 0.011)	Loss 2.2225e+00 (2.3077e+00)	Acc@1  15.62 ( 15.10)	Acc@5  70.31 ( 64.00)
Epoch: [0][60/79]	Time  2.364 ( 2.350)	Data  0.000 ( 0.010)	Loss 1.9495e+00 (2.2652e+00)	Acc@1  26.56 ( 16.29)	Acc@5  76.56 ( 65.60)
Epoch: [0][70/79]	Time  2.367 ( 2.352)

In [ ]:
import os
model_size = (os.path.getsize(SAVEPATH+'model_weightz.pth')) / 1024.

print(f'Your model size is {model_size} KiB.')
if model_size > 300.0:
    print('It is over 300 KiB!!')
    print('Please more compress your model..')

Your model size is 255.791015625 KiB.


In [ ]:
import torch
import pandas as pd
import argparse
import time
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


def evaluation():
    # Data loading PART (If you want to change for your model, you can change this part..)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    test_dataset = torchvision.datasets.ImageFolder('./test', transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCHSIZE, num_workers=4, shuffle=False)

    # Model loading PART (If you want to change for loading your model, you can change this part..)
    model = Model()
    model = model.cuda()
    model.load_state_dict(torch.load(SAVEPATH+'model_weightz.pth'))

    # Making an evaluation csv file PART (DON'T CHANGE BELOW EXCEPT WHEN NECESSARY!!)
    print('Make an evaluation csv file for kaggle submission...')
    model.eval()

    Category = []
    for input, _ in test_loader:
        input = input.cuda()
        output = model(input)
        output = torch.argmax(output, dim=1)
        Category = Category + output.tolist()

    Id = list(range(0, 8000))
    samples = {
       'Id': Id,
       'Category': Category 
    }
    df = pd.DataFrame(samples, columns=['Id', 'Category'])

    df.to_csv(SAVEPATH+'submission.csv', index=False)
    print('Done!!')


if __name__ == "__main__":
    import os
    model_size = (os.path.getsize(SAVEPATH+'model_weightz.pth')) / 1024.
    print(f'Your model size is {model_size} KiB.')
    if model_size > 300.0:
        print('It is over 300 KiB!!\n')
        print('Please more compress your model..')
    else:
        print('It is under 300 KiB.\n')
        evaluation()

Your model size is 255.791015625 KiB.
It is under 300 KiB.

Make an evaluation csv file for kaggle submission...
Done!!
